<a href="https://colab.research.google.com/github/earo12/BBVA_ETL/blob/dev-branch/Examen_tecnico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 1

In [ ]:
# Pregunta 1

# He ejecutado el primer dataframe como csv
# Importamos librerias necesarias
import pandas as pd



In [ ]:
# Elimina las transacciones con status "Failed"
# Usualmente, el estándar es denominar dataframes como "df"
df = pd.read_csv('/content/ejercicio1 - Hoja 1.csv')


# Eso lo logramos con filtrar aquellas filas cuyo estatus no sea Failed i.e. usando el operdador !=
df = df[df['status'] != 'Failed']
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/ejercicio1 - Hoja 1.csv'

In [ ]:
# Rellena los valores nulos con la columna transaction amount con el promedio de valores existentes
# Esto es, vamos a rellenar los valores nulos con el promedio de transaction_amount
df = df.fillna(df['transaction_amount'].mean())

In [ ]:
df

,transaction_id,customer_id,product_type,transaction_amount,status
0,1,1001,Loan,1500.0,Completed
1,2,1002,Deposit,2000.0,Completed
2,3,1003,CreditCard,2500.0,Pending
3,4,1004,Loan,2000.0,Completed


In [ ]:
# Genera un resumen que muestre el total de transacciones por product_type
# Esto es básicamente usar un group by, por product_type y aplicar un sum:

df_resumido = df.groupby(['product_type'])['transaction_amount'].sum()
df_resumido

,transaction_amount
product_type,
CreditCard,2500.0
Deposit,2000.0
Loan,3500.0


In [ ]:
# Como podemos ver, tenemos 2500 para CreditCard, 2000 para Deposits, 3500 para Loans

In [ ]:
# Guarda los datos limpios y el resumen en dos archivos CSV separados:
# Si tomtamos como df_resumido, i.e., al que le hicimos el groupby en el paso anterior, necesitamos aplicar el método .to_csv() para lograrlo
df_limpio = df_resumido.to_csv('df_limpio.csv')
# Esto nos generará un archivo .csv en Colab que podemos descargar

NameError: name 'df_resumido' is not defined

# Ejercicio 2



In [ ]:
# Nuevamente, ejecutamos el dataframe  desde local:

df = pd.read_csv('/content/ejercicio2 - Hoja 1.csv')

In [ ]:
df

,transaction_id,customer_id,product_type,transaction_amount,status
0,1,1001,Loan,1500.0,Completed
1,2,1002,Deposit,NaN,Pending
2,2,1002,Deposit,NaN,Pending
3,3,1003,CreditCard,2500.0,Completed
4,4,1004,Loan,NaN,Pending


In [ ]:
# Identifica y eliminina los duplicados en trasnaction_id

# Eso se logra con el método drop_duplicates:
df_sin_dups = df.drop_duplicates('transaction_id')
df_sin_dups

,transaction_id,customer_id,product_type,transaction_amount,status
0,1,1001,Loan,1500.0,Completed
1,2,1002,Deposit,NaN,Pending
3,3,1003,CreditCard,2500.0,Completed
4,4,1004,Loan,NaN,Pending


In [ ]:
# Rellena los valores  nulos de la columna transction_amount con los valores promedio de los valores existentes:

df_sin_nulos = df_sin_dups.fillna(df_sin_dups['transaction_amount'].mean())

df_sin_nulos

,transaction_id,customer_id,product_type,transaction_amount,status
0,1,1001,Loan,1500.0,Completed
1,2,1002,Deposit,2000.0,Pending
3,3,1003,CreditCard,2500.0,Completed
4,4,1004,Loan,2000.0,Pending


In [ ]:
# Actualiza las transacctiones con estatus "Pending" tengan un estatus "Processing":
# El método .replace() nos ayuda, el primer argumento es el valor "viejo" y el segundo, el valor "nuevo"

df_actualizado = df_sin_nulos.replace('Pending', 'Processing')
df_actualizado

,transaction_id,customer_id,product_type,transaction_amount,status
0,1,1001,Loan,1500.0,Completed
1,2,1002,Deposit,2000.0,Processing
3,3,1003,CreditCard,2500.0,Completed
4,4,1004,Loan,2000.0,Processing


# Parte 2

## PySpark

In [ ]:
!pip install PySpark

In [ ]:
# Importamos la sesión de Spark
from pyspark.sql import SparkSession
# También creamos la app con la que trabajamos en Spark
sp = SparkSession.builder.appName('ejemplo').getOrCreate()

# El paso anterior es crucial dado que, de otra forma, no podemos trabajar con Spark

In [ ]:
# Leemos el dataframe de Parte2_PySpark con spark, establecemos el header como True para que nos tome la primera columna como encabezado
df_spark = sp.read.csv('/content/Parte2_PySpark - Hoja 1.csv', header=True)

In [ ]:
# podemos aplicar un show para ver que efectivamente lo ejecutamos bien, CUIDADO!!! , Spark es  Lazy, esto solo es de prueba, no se recomeinda hacerlo:
# En un entorno productivo, es recomendable no descomentar esta línea
#df_spark.show()

+--------------+-----------+------------+------------------+---------+
|transaction_id|customer_id|product_type|transaction_amount|   status|
+--------------+-----------+------------+------------------+---------+
|             1|       1001|        Loan|              1500|Completed|
|             2|       1002|     Deposit|              2000|Completed|
|             3|       1003|  CreditCard|              2500|  Pending|
|             4|       1004|        Loan|              3000|Completed|
|             5|       1005|     Deposit|              1000|   Failed|
+--------------+-----------+------------+------------------+---------+



In [ ]:
# Reparticiona el dataframe con product_type para optimizar el procesamiento:
# esto se logra con el método .repartition
df_particionado = df_spark.repartition('product_type')

In [ ]:
# Guarda los datos particionados en formato Parquet
# podemos usar el método .write seguido del método .parquet, esto descargará directamente el archivo en Colab
df_particionado.write.parquet('df_particionado')

In [ ]:
# Dado el siguiente DataFrame en PySpark:

data = [('A', 100), ('B', 200), ('A', 300), ('B', None)]
columns = ['product_type', 'amount']
df = sp.createDataFrame(data, columns)

In [ ]:
# Escribe el código para reemplazar valores nulos con 0
df = df.fillna(0)

# Agrupar por product_type
df_agrupado = df.groupBy('product_type')

# Calcular la suma por amount:

df_agrupado = df_agrupado.sum('amount').show()

+------------+-----------+
|product_type|sum(amount)|
+------------+-----------+
|           B|        200|
|           A|        400|
+------------+-----------+



In [ ]:
# Antes que nada, conviene trarese la función col del conjunto de funciones de pyspark
from pyspark.sql.functions import col

In [ ]:
# Pregunta práctica avanzada

#df = df.withColumn('new_col', col('existing_col') * 10).filter(col('existing_col') > 1000)



# Optimízalo para mejorar su rendimiento.
# Una forma que podemos hacer, asumiendo que estamos trabajando en un área productiva es aumentar las particiones de Spark
# Spark trabaja con particiones con clústeres, cuando trabajamos con un dataframe, los clústeres leen esa información, lo que se vuelve sencillo si tenemos menos carga de trabajo en lugar de mandar todo el dataframe de golpe


# Asumimos que "existing_col" es el valor de amount del dataframe anterior

df = df.withColumn('nueva_columna', col('amount') * 10).filter(col('amount') > 1000)
#df.show()
# Importante, si trabajamos en un entorno que cargue un csv por ejemplo, el usar el método repartition, nos generará más archivos csv como salida


df_nuevo = df.repartition(1000)


# Parte 3
## SQL


In [ ]:
# La tabla 1 de informacion de transacciones
# La tabla 2 tiene información de customers:
# Para la tabla 1 tenemos lo siguiente, podemos crear tablas temporales con los dataframes si los trabajamos con Spark:
df_customers = sp.read.csv('/content/Ejercicio4_TablaCustomers - Hoja 1.csv', header=True)
df_transactions = sp.read.csv('/content/Ejercicio4_TablaTransactions - Hoja 1.csv', header=True)

In [ ]:
df_customers.createOrReplaceTempView('customers')
df_transactions.createOrReplaceTempView('transactions')

In [ ]:
# El ejecutar dicho código, nos permite ejecutar sentencias SQL como usualmente estamos acostumbrados, por ejemplo
query = """SELECT * FROM transactions LIMIT 5
        """
sp.sql(query).show()

+--------------+-----------+------------------+----------+
|transaction_id|customer_id|transaction_amount|      date|
+--------------+-----------+------------------+----------+
|           101|       1001|              1500|2024-01-01|
|           102|       1002|              2000|2024-01-02|
|           103|       1003|              2500|2024-01-03|
|           104|       1004|              3000|2024-01-04|
|           105|       1005|              1000|2024-01-05|
+--------------+-----------+------------------+----------+



In [ ]:
# Genera un reporte que combinado que muestre las tablas para mostrar nombre_cliente, region, monto_total_tranascciones, numero_transacciones_realizadas
# EL campo de numero de transacciones realizadas no está en las tablas como transaction_count, sin embargo, se puede realizar con un SUM(1) o un COUNT(*) y agurpar
# Adicional, le doy prioridad a la tabla de clientes para hacer el join respectivo:
query = """
          SELECT cc.customer_name AS nombre_cliente,
                 cc.region AS region,
                 tt.transaction_amount AS monto_total_transacciones,
                 SUM(1) AS numero_transacciones_realizadas
          FROM customers AS cc
          LEFT JOIN transactions AS tt
          ON cc.customer_id = tt.customer_id
          GROUP BY cc.customer_name, cc.region, tt.transaction_amount;
        """

sp.sql(query).show()

+--------------+------+-------------------------+-------------------------------+
|nombre_cliente|region|monto_total_transacciones|numero_transacciones_realizadas|
+--------------+------+-------------------------+-------------------------------+
|         David|  East|                     3000|                              1|
|           Bob| South|                     2000|                              1|
|           Eva| South|                     1000|                              1|
|         Alice| North|                     1500|                              1|
|       Charlie| North|                     2500|                              1|
+--------------+------+-------------------------+-------------------------------+



In [ ]:
# Filtra aquellos valores con valor "North"
# Igual aplicamos el metodo de creacion de tabla temporal, para aplicar el WHERE como filtrado
df = sp.sql(query)

df.createOrReplaceTempView('reporte')

query_filtrado = """
                  SELECT nombre_cliente,
                        region,
                        monto_total_transacciones,
                        numero_transacciones_realizadas
                  FROM reporte
                  WHERE region = 'North'
                """

sp.sql(query_filtrado).show()


+--------------+------+-------------------------+-------------------------------+
|nombre_cliente|region|monto_total_transacciones|numero_transacciones_realizadas|
+--------------+------+-------------------------+-------------------------------+
|         Alice| North|                     1500|                              1|
|       Charlie| North|                     2500|                              1|
+--------------+------+-------------------------+-------------------------------+



# Ejercicio 5
## ETL

In [ ]:
# Parte de extracción:

df_extraccion = sp.read.csv('/content/Ejercicio5_ETL - Hoja 1.csv', header=True)


In [ ]:
# Parte de transformación:
from pyspark.sql.functions import col, when

df_extraccion = df_extraccion.fillna('0')
df_extraccion.createOrReplaceTempView('clientes')

In [ ]:
df_extraccion.show()

+-----------+-------------+------+---------------+
|customer_id|customer_name|region|account_balance|
+-----------+-------------+------+---------------+
|       1001|        Alice| North|           1500|
|       1002|          Bob| South|           2000|
|       1003|      Charlie| North|              0|
|       1004|        David|  East|           1800|
|       1005|          Eva| South|            500|
+-----------+-------------+------+---------------+



In [ ]:
query = """
            SELECT customer_id,
                   customer_name,
                   region,
                   account_balance,
                   CASE WHEN account_balance < 1000 THEN 'Low'
                        WHEN account_balance >= 1000 AND account_balance < 3000 THEN 'Medium'
                        WHEN account_balance >= 3000 THEN 'High'
                        END AS balance_category
            FROM clientes;
        """
df_transformado = sp.sql(query)

In [ ]:
df_transformado.createOrReplaceTempView('customers_transformed')

In [ ]:
query = """ SELECT * FROM customers_transformed LIMIT 5;
        """
sp.sql(query).show()

+-----------+-------------+------+---------------+----------------+
|customer_id|customer_name|region|account_balance|balance_category|
+-----------+-------------+------+---------------+----------------+
|       1001|        Alice| North|           1500|          Medium|
|       1002|          Bob| South|           2000|          Medium|
|       1003|      Charlie| North|              0|             Low|
|       1004|        David|  East|           1800|          Medium|
|       1005|          Eva| South|            500|             Low|
+-----------+-------------+------+---------------+----------------+

